In [1]:
from PIL import Image, ImageDraw, ImageFont
import random
import cv2
import os
import matplotlib.pyplot as plt

In [12]:
def num_to_color(xor_num):
    # Simple mapping function to convert XOR result to RGB values
    r = xor_num % 256
    g = (xor_num // 256) % 256
    b = (xor_num // 65536) % 256
    return r, g, b

def draw_color_maze(image, draw, maze_size, color, user_id, number, savegif=False):
    # Set a fixed seed for the random number generator based on user ID and number
    random.seed(hash((user_id, number)))
    
    if savegif == True:
        counter = 1
    # Use the chosen color for the maze walls
    for i in range(0, maze_size[0], 10):
        for j in range(0, maze_size[1], 10):
            if random.random() < 0.4:  # Adjust this probability to control maze density
                draw.rectangle([i, j, i + 10, j + 10], fill=color)
                if savegif == True:
                    print('herhe')
                    file_name = os.path.join(os.getcwd(), 'maze_steps', f'{counter}.png')
                    image.save(file_name)
                    counter += 1

def create_color_maze_image_with_xor(user_id, number, output_path='color_maze_image.png', maze_size=(252, 252), font_size=30):
    # XOR the user ID and the number
    xor_result = user_id ^ number

    # Map the XOR result to RGB values
    bg_color = num_to_color(xor_result)
    maze_color = num_to_color(user_id + 50)
    
    # Create a blank white image
    image = Image.new('RGB', maze_size, bg_color)
    draw = ImageDraw.Draw(image)

    # Draw the color maze pattern
    draw_color_maze(image, draw, maze_size, maze_color, user_id, number)

    # Save the image
    image.save(output_path)

In [23]:
# Manually change places at location 1, location 2, loaction 3 to generate multiple key for each user
# user_ids = [102211000, 202222000, 131122000, 123100, 134200, 391100]
user_id = 391100 # location 1
with open('biohash_numbers_K1.txt', 'r') as f: # location 2
    for biohash_index, line in enumerate(f.readlines(), start=user_id):
        
        # Read the biohash number
        biohash_number = line
        biohash_number = int(biohash_number.split('\n')[0])
        # long_biohash_number = ''.join([biohash_number]*4)
        
        # print(biohash_number, biohash_index)
        dir_name = os.path.join(os.getcwd(), 'maze_dataset', 'k1', str(biohash_number)) # location 3
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        file_name1 = os.path.join(dir_name, '1.png')
        file_name2 = os.path.join(dir_name, '2.png')
        
        create_color_maze_image_with_xor(user_id=biohash_index, number=biohash_number, output_path=file_name1)
        create_color_maze_image_with_xor(user_id=biohash_index, number=biohash_number, output_path=file_name2)